In [1]:
import torchvision
import torch
from torch.utils.data import DataLoader
from torchvision.datasets.folder import ImageFolder
from torchvision import transforms
from torchvision import models
import torch.nn as nn
from torch import optim

batch_size = 32
nb_categories = 101

In [2]:
torch.__version__

'1.0.1'

In [2]:
data_root = '/home/wenjian/data/'

train_set_dir = data_root + 'wiki_crop_train/'
val_set_dir = data_root + 'wiki_crop_val/'
test_set_dir = data_root + 'wiki_crop_test/'

In [3]:
train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                                [0.229, 0.224, 0.225])]) # Imagenet standards

val_transform = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                                [0.229, 0.224, 0.225])]) # Imagenet standards

# What should we do for test??????
test_transform = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor()])

train_set = ImageFolder(train_set_dir, transform=train_transform, target_transform=None)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

val_set = ImageFolder(val_set_dir, transform=val_transform, target_transform=None)
val_loader = DataLoader(val_set, batch_size=batch_size)

test_set = ImageFolder(test_set_dir, transform=test_transform, target_transform=None)
test_loader = DataLoader(test_set, batch_size=batch_size)

In [10]:
input('Sure to load model 1? Press enter to continue')
model = models.vgg16(pretrained=True)
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False

class DotWithConstantVector(torch.nn.Module):
    def __init__(self, c):
        super(DotWithConstantVector, self).__init__()
        self.c = c
        
    def forward(self, x):
        return torch.matmul(x, self.c)
        #return torch.bmm(x, c)

value_vector = torch.ones((101, 1))
value_vector = value_vector.new_tensor(range(101))


# The original classifier[6]'s input features
n_inputs = model.classifier[3].out_features
# Add on classifier
model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 512), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(512, 101),                   
                      nn.Softmax(),
                      DotWithConstantVector(value_vector))
model.summary()

# Loss and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

n_epochs = 10
model.train()

import time



for epoch in range(n_epochs):
    
    time_start = time.time()
    
    for batch_id, (data, label) in enumerate(train_loader):
        print('batch_id=', batch_id, '  ---label--- ', label)
        # Generate predictions
        out = model(data)
        # Calculate loss
        #print('---out--- ', out)
        label = label.float()
        loss = criterion(out, label)
        print('loss=', loss)
        # Backpropagation
        loss.backward()
        # Update model parameters
        optimizer.step()
    time_end = time.time()
    torch.save(model.state_dict(), '/home/wenjian/results')
    with open('/home/wenjian/results/mylog.txt', 'a') as f:
        f.write('epoch '+str(epoch)+' finished. Time consumed: ', time_end-time_start)

KeyboardInterrupt: 

In [4]:
model2 = models.vgg16(pretrained=True)
for param in model2.parameters():
    param.requires_grad = False
# The original classifier[6]'s input features
n_inputs = model2.classifier[3].out_features
# Add on classifier
model2.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 512), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(512, nb_categories),                   
                      nn.Softmax())

Question: unbalanced dataset weights for loss function

In [ ]:

def cross_entropy_between_distributions(pred, target):
    # The softmax part is not included in this function
    entropy = 0.0
    assert pred.shape != target.shape, 'shapes not match for cross_entropy_between_distributions'
    for batch_index in range(pred.shape[0]):
        for i in range(pred.shape[1]):
            entropy += target[batch_index, i]*torch.log(pred[batch_index, i])
    return entropy
    

In [5]:
criterion = nn.KLDivLoss()
optimizer = optim.Adam(model2.parameters())

In [ ]:
   
def from_one_hot(vector, gap=0.1):
    n = argmax(vector)
    return np.array(linear_shadow(n, gap))

In [8]:
def linear_shadow(label_batch, gap=0.2, nb_categories=101):
    batch_size = label_batch.shape[0]
    l = torch.zeros(batch_size, nb_categories, dtype= torch.float)
    for i in range(batch_size):
        label = torch.tensor(label_batch[i], dtype=torch.float)
        for j in range(nb_categories):
            l[i,j] = 1- abs(label-j)*gap
            if l[i,j]<0:
                l[i,j]=0
    return l


In [20]:
def target_distribution(vectors):
    # To normalize the distribution. Shape of the vectors are expected as (batch_size, vector_dim)
    # Noted that the softmax function in Pytorch only work for float type tensor
    return nn.functional.softmax(vectors, dim=1)

In [30]:
n_epochs = 10
model2.train()

import time



for epoch in range(n_epochs):
    
    time_start = time.time()
    
    running_loss = 0.0
    #running_corrects = 0
    
    for batch_id, (data, labels) in enumerate(train_loader):
        print('batch_id=', batch_id, '  ---labels--- ', labels)
        # Generate predictions
        out = model2(data)
        log_proba = torch.log(out)
        # Calculate loss
        #print('---out--- ', out)
        
        print('label.shape ', labels.shape)
        labels = linear_shadow(labels)
        labels = target_distribution(labels) # Normalize th distribution
        print('labels', labels)
        #labels = labels.float()
        loss = criterion(log_proba, labels)
        print('loss=', loss)
        # Backpropagation
        loss.backward()
        # Update model parameters
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()*data.size(0)
        #running_corrects += torch.sum(preds == labels.data)
        
    time_end = time.time()
    torch.save(model.state_dict(), '/home/wenjian/results2')
    with open('/home/wenjian/results/mylog2.txt', 'a') as f:
        f.write('epoch '+str(epoch)+' finished. Time consumed: ', time_end-time_start)

batch_id= 0   ---labels---  tensor([18, 20, 45, 19, 14, 18, 22, 19, 32, 33, 64, 21, 25, 47, 94, 18, 30, 67,
        22, 27, 26, 70, 26, 16,  6, 18, 10, 50, 25, 77, 23, 15])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        ...,
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092]])
loss= tensor(0.0142, grad_fn=<KlDivBackward>)


/home/wenjian/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


batch_id= 1   ---labels---  tensor([26, 50, 37, 40, 55, 21, 67, 17, 22, 27, 14, 26, 14, 22, 14, 16, 54, 32,
        30, 21, 23, 45, 50,  6, 14, 21, 37, 18, 30, 20, 59, 12])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        ...,
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092]])
loss= tensor(0.0100, grad_fn=<KlDivBackward>)
batch_id= 2   ---labels---  tensor([63, 29, 51, 12, 19, 48, 49, 30, 15, 53, 20, 27, 21, 41, 22, 45,  6, 16,
        44, 15, 22, 41, 61, 30, 62, 16, 15, 57, 27, 44, 37, 34])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        

batch_id= 14   ---labels---  tensor([23, 30, 63, 11, 29, 71, 15, 80, 18, 21, 19,  9, 53, 19, 95, 18, 17, 37,
        47, 26, 81, 75, 16, 38, 22, 47, 16, 53, 19, 36, 31, 18])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        ...,
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092]])
loss= tensor(0.0020, grad_fn=<KlDivBackward>)
batch_id= 15   ---labels---  tensor([40, 16, 31, 52, 14, 16, 20, 44, 61, 29, 85, 16, 34, 27, 16, 52, 26, 21,
        21, 53, 20, 31, 30, 65, 20, 19, 22, 21, 60, 15, 14, 17])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
      

batch_id= 27   ---labels---  tensor([16, 22, 80, 23, 44, 33, 17, 36, 23, 22, 43, 15, 28, 17, 60, 15, 16, 61,
        32, 59, 52, 75, 19, 54, 54, 44, 17, 21, 23, 41, 66, 41])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        ...,
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092]])
loss= tensor(0.0003, grad_fn=<KlDivBackward>)
batch_id= 28   ---labels---  tensor([50, 33, 56, 20, 43, 47,  6, 26, 16, 21, 16, 19, 21, 27, 23, 36, 29, 36,
        31, 16, 36, 18, 31, 22, 28, 17, 16, 22, 82, 21, 18, 30])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
      

batch_id= 40   ---labels---  tensor([70, 31, 38, 51, 92, 29, 26, 12, 14, 21, 20, 30, 19, 62, 38, 18, 31, 52,
        15, 18, 17, 36, 66, 21, 27, 14, 77, 28, 32, 31, 17, 34])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        ...,
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092]])
loss= tensor(0.0003, grad_fn=<KlDivBackward>)
batch_id= 41   ---labels---  tensor([87, 83, 21, 63, 20, 52, 31, 20, 44, 14, 37, 61, 25, 65, 40, 73, 20, 12,
        27, 67, 23, 28, 42, 27, 17, 19, 77, 84, 22, 36, 32, 17])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
      

batch_id= 53   ---labels---  tensor([47, 39, 23, 10, 22, 30, 25, 15, 38, 28, 30, 20, 33, 11, 26, 31, 21, 23,
        60, 29, 22, 18, 18, 18, 14, 18, 22,  8, 62, 41, 12, 27])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        ...,
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092]])
loss= tensor(0.0004, grad_fn=<KlDivBackward>)
batch_id= 54   ---labels---  tensor([19, 19, 22, 27, 18, 75, 21, 28, 15, 16, 15, 32, 62, 56,  9, 31, 19, 33,
        20, 27, 41, 18, 17, 12, 23, 42, 49, 49, 47, 82, 38, 28])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
      

batch_id= 66   ---labels---  tensor([22, 16, 17, 17, 33, 26, 63, 15, 45, 58, 36, 40, 11, 15, 54, 38, 47, 20,
        22, 21, 26, 16, 36, 63, 33, 10, 42, 78, 56, 40, 14, 21])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        ...,
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092]])
loss= tensor(0.0003, grad_fn=<KlDivBackward>)
batch_id= 67   ---labels---  tensor([32, 69, 26, 14, 20, 24, 23, 54, 23, 47, 69, 43, 12, 47, 38, 22, 86, 27,
        42, 21, 27, 44, 20, 20, 28, 22, 48, 58, 26, 37, 61, 17])
label.shape  torch.Size([32])
labels tensor([[0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
        [0.0092, 0.0092, 0.0092,  ..., 0.0092, 0.0092, 0.0092],
      

KeyboardInterrupt: 

In [3]:
input2 = torch.randn(3, 5, requires_grad=True)

In [4]:
input2

tensor([[ 0.9042,  0.1765,  0.5229,  0.2021,  0.5831],
        [-0.5932,  0.9197,  0.0721, -1.7064,  0.4047],
        [ 0.1000,  0.2682, -1.6858,  0.5479,  1.1514]], requires_grad=True)

In [5]:
input2.shape

torch.Size([3, 5])

In [6]:
len(input2)

3

In [13]:
tensor_try = torch.tensor([77, 51, 25, 2, 20, 14, 19, 21, 38, 26,  8, 61, 31, 17, 62, 32, 22, 15,
        26, 42, 23, 30, 69, 27, 36, 49, 47,  9, 37, 86, 36, 54])

In [14]:
tensor_try

tensor([77, 51, 25,  2, 20, 14, 19, 21, 38, 26,  8, 61, 31, 17, 62, 32, 22, 15,
        26, 42, 23, 30, 69, 27, 36, 49, 47,  9, 37, 86, 36, 54])

In [15]:
a = linear_shadow(tensor_try)

/home/wenjian/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [16]:
a[3]

tensor([0.6000, 0.8000, 1.0000, 0.8000, 0.6000, 0.4000, 0.2000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])

In [3]:
loss = nn.CrossEntropyLoss()
inputs = torch.randn(3, 5, requires_grad=True)
print(inputs)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(inputs, target)
print(output)
output.backward()

tensor([[ 0.3651,  1.9261, -1.6455, -1.2310,  1.0050],
        [ 0.2845,  0.0487, -0.1906,  1.8214, -0.8511],
        [ 0.9083,  0.2055, -0.2629, -0.2203, -2.3165]], requires_grad=True)
tensor([1, 4, 0])
tensor(1.4756, grad_fn=<NllLossBackward>)


In [26]:
a = torch.tensor([[1,2,3,4],[1,1,1,1]], dtype=torch.float)

In [27]:
a

tensor([[1., 2., 3., 4.],
        [1., 1., 1., 1.]])

In [29]:
target_distribution(a)

tensor([[0.0321, 0.0871, 0.2369, 0.6439],
        [0.2500, 0.2500, 0.2500, 0.2500]])

In [28]:
a.softmax(dim=1)

tensor([[0.0321, 0.0871, 0.2369, 0.6439],
        [0.2500, 0.2500, 0.2500, 0.2500]])

In [1]:
a = [1,2,3]
b = [4,5,6]

In [2]:
a+b

[1, 2, 3, 4, 5, 6]